In [4]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
import sys
sys.path.append('/content/gdrive/MyDrive/6893/')

In [6]:
import numpy as np
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import gensim
from gensim.models import Word2Vec
from sklearn.metrics import classification_report
import re

In [7]:
df = pd.read_csv("/content/gdrive/MyDrive/6893/tidydata.csv")
df['label'] = df['recommended'].apply(lambda x: 0 if x== True else 1)
X = df[['review']]
X['review']=X['review'].apply(lambda x: np.str_(x))
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(168002, 1)
(42001, 1)
(168002,)
(42001,)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [12]:
X_train.head()

,review
30220,head bobbing feature give motion sickness opti...
51343,boring gameplay
151799,cute
58107,i 'd rather play fortnite
122153,rating 10/10 intended audience 閳斤拷 kid 閳斤拷 eve...


### tf

In [8]:
vector = TfidfVectorizer(max_features=1000)
train_tfidf = vector.fit_transform(X_train['review'].apply(lambda x: np.str_(x)))
test_tfidf = vector.transform(X_test['review'].apply(lambda x: np.str_(x)))
df_tfidf_train = pd.DataFrame(train_tfidf.todense())
df_tfidf_test = pd.DataFrame(test_tfidf.todense())
print(df_tfidf_train.shape)
print(df_tfidf_test.shape)

(168002, 1000)
(42001, 1000)


Enhanced features

n-grams

In [28]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [29]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.corpus import stopwords
from sklearn.feature_selection import SelectKBest
en_stopwords = set(stopwords.words("english"))
cout = CountVectorizer(ngram_range=(1,3), analyzer='word', lowercase=True, stop_words=en_stopwords)
vec_train_ngram = cout.fit_transform(X_train['review'].apply(lambda x: np.str_(x)))
#vec_train_ngram = cout.transform().toarray()
vec_test_ngram = cout.transform(X_test['review'].apply(lambda x: np.str_(x)))
print(vec_train_ngram.shape)
print(vec_test_ngram.shape)

(168002, 5968945)
(42001, 5968945)


In [54]:
skb = SelectKBest(k=600).fit(vec_train_ngram, y_train)
train_ngram = skb.transform(vec_train_ngram)
test_ngram = skb.transform(vec_test_ngram)
print(train_ngram.shape)
print(test_ngram.shape)
df_ngram_train = pd.DataFrame(train_ngram.todense())
df_ngram_test = pd.DataFrame(test_ngram.todense())

(168002, 600)
(42001, 600)


Lexicon-based feature

In [ ]:
from tqdm import tqdm
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
import os
import glob
import time
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import word_tokenize
import re

In [ ]:
def preprocess_lexicon(uni_txt, bi_txt, train_set, test_set):
    glob.glob("lexica/*/*")
    d_uni = {}
    for i in [i.strip() for i in open(uni_txt).readlines()]:
        d_uni[i.split("\t")[0]] = i.split("\t")[1:]
    d_bi = {}
    # for i in [i.strip() for i in open(bi_txt).readlines()]:
    #     d_bi[i.split("\t")[0]] = i.split("\t")[1:]

    train_14feature = preprocess_check_data_in_lexicon(train_set, d_uni, d_bi)
    #train_14feature=[[]]
    #print('test_set shape is,',test_set.shape)
    test_14feature = preprocess_check_data_in_lexicon(test_set, d_uni, d_bi)
    print('test_14feature has the shape,', test_14feature.shape)
    return train_14feature, test_14feature
def preprocess_check_data_in_lexicon(dataset, d_uni,d_bi):
    unilist = d_uni.keys()
    un_usekey = {}
    for i in tqdm(list(dataset["review"])):
      wordlist = i.split()
      for word in wordlist:
        if word in unilist:
          un_usekey[word] = d_uni[word]
        # for key in d_uni.keys():
        #     if key in i:
        #         un_usekey[key] = d_uni[key]
        else:
               pass
    bi_usekey = {}
    # for i in tqdm(list(dataset["review"])):
    #     for key in d_bi.keys():
    #         if key in i:
    #             bi_usekey[key] = d_bi[key]
    #         else:
    #             pass
    return get_feature_vec(dataset, un_usekey, bi_usekey)
def get_feature_vec(dataset, un_usekey, bi_usekey):

    un_vec = getother_feature_gram(un_usekey, list(dataset["review"]), sign="un")
    print('unvec shape is ', un_vec.shape)
    # bi_vec = getother_feature_gram(bi_usekey,list(dataset["review"]), sign="bi")
    # vec_lexfeature = np.concatenate([un_vec,bi_vec],axis=1)
    #print('vec_lexfeature shape is ', vec_lexfeature.shape)
    print('vec_lexfeature shape is ', un_vec.shape)
    return un_vec

def getother_feature_gram(d_bi, dfall, sign="bi"):
    pos_token_num = []
    neg_token_num = []
    score_all = []
    pos_max_all = []
    neg_max_all = []
    neg_last_all = []
    pos_last_all = []
    for key in d_bi.keys():
        d_bi[key][0] = float(d_bi[key][0])
    for i in tqdm(dfall):
        pos_token = 0
        neg_token = 0
        score = 0
        pos_max = 0
        neg_max = 0

        for key in d_bi.keys():
            if key in i:
                if d_bi[key][0] >= 0:
                    pos_token += 1
                else:
                    neg_token += 1
                score += d_bi[key][0]
                pos_max = max(pos_max, d_bi[key][0])
                neg_max = min(neg_max, d_bi[key][0])
        pos_token_num.append(pos_token)
        neg_token_num.append(neg_token)
        score_all.append(score)
        pos_max_all.append(pos_max)
        neg_max_all.append(neg_max)
        pos_last = 0
        neg_last = 0
        if sign == "un":
            i = i.split(" ")
            for z in i[::-1]:
                if z in d_bi.keys():
                    if d_bi[z][0] >= 0:
                        pos_last = d_bi[z][0]
                        break
            for z in i[::-1]:
                if z in d_bi.keys():
                    if d_bi[key][0] < 0:
                        neg_last = d_bi[key][0]
                        break
        elif sign == "bi":
            i = i.split(" ")
            for z in reversed(range(len(i))):
                if z - 1 >= 0:
                    key = " ".join(i[z - 1:z + 1])
                    if key in d_bi.keys():
                        if d_bi[key][0] >= 0:
                            pos_last = d_bi[key][0]
                            break
            for z in reversed(range(len(i))):
                if z - 1 >= 0:
                    key = " ".join(i[z - 1:z + 1])
                    if key in d_bi.keys():
                        if d_bi[key][0] < 0:
                            neg_last = d_bi[key][0]
                            break

        neg_last_all.append(neg_last)
        pos_last_all.append(pos_last)
    pos_token_num = np.array(pos_token_num).reshape(-1, 1)
    neg_token_num = np.array(neg_token_num).reshape(-1, 1)
    score_all = np.array(score_all).reshape(-1, 1)
    pos_max_all = np.array(pos_max_all).reshape(-1, 1)
    neg_max_all = np.array(neg_max_all).reshape(-1, 1)
    neg_last_all = np.array(neg_last_all).reshape(-1, 1)
    pos_last_all = np.array(pos_last_all).reshape(-1, 1)
    return np.concatenate([pos_token_num, neg_token_num, score_all, pos_max_all, neg_max_all, neg_last_all, pos_last_all], axis=1)

In [ ]:
uni_txt = "/content/gdrive/MyDrive/6893/Emoticon-unigrams.txt"
bi_txt = "/content/gdrive/MyDrive/6893/Emoticon-bigrams.txt"
train_set, test_set = X_train, X_test
train_lexicon, test_lexicon = preprocess_lexicon(uni_txt,bi_txt,train_set,test_set)

100%|██████████| 168002/168002 [15:03<00:00, 185.99it/s]


unvec shape is  (168002, 7)
vec_lexfeature shape is  (168002, 7)


100%|██████████| 42001/42001 [03:03<00:00, 229.33it/s]

unvec shape is  (42001, 7)
vec_lexfeature shape is  (42001, 7)
test_14feature has the shape, (42001, 7)


In [31]:
df_lexicon_train = pd.DataFrame(train_lexicon)
df_lexicon_test = pd.DataFrame(test_lexicon)
print(df_lexicon_train.shape)
print(df_lexicon_test.shape)
df_lexicon_train.to_csv('/content/gdrive/MyDrive/6893/df_lexicon_train.csv')
df_lexicon_test.to_csv('/content/gdrive/MyDrive/6893/df_lexicon_test.csv')

NameError: ignored

In [32]:
df_lexicon_train= pd.read_csv('/content/gdrive/MyDrive/6893/df_lexicon_train.csv')
df_lexicon_test=pd.read_csv('/content/gdrive/MyDrive/6893/df_lexicon_test.csv')

In [56]:
df_enhanced_train = pd.concat([df_ngram_train,df_lexicon_train],axis=1)
df_enhanced_test = pd.concat([df_ngram_test,df_lexicon_test],axis=1)
df_enhanced_train = df_ngram_train
df_enhanced_test = df_ngram_test
print(df_enhanced_train.shape)
print(df_enhanced_test.shape)

(168002, 600)
(42001, 600)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,75.0,64.0,8.930,5.000,-1.651,-0.676,0.045,12.0,11.0,-4.921,0.742,-1.454,0.000,0.049
1,116.0,106.0,3.896,1.658,-4.999,-0.676,0.854,26.0,30.0,-9.282,1.658,-1.896,-1.742,0.378
2,149.0,125.0,5.281,5.000,-4.999,-0.676,0.359,45.0,40.0,-16.603,1.840,-4.999,-1.644,0.023
3,20.0,21.0,4.089,1.995,-0.418,-0.676,0.929,5.0,2.0,-0.229,1.289,-1.203,0.000,0.000
4,19.0,20.0,1.258,0.589,-0.692,-0.676,0.000,3.0,2.0,-0.353,0.742,-0.906,-0.906,0.000


## Model

### Naive Bayes

In [57]:
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(df_enhanced_train, y_train)
print(classification_report(gnb.predict(df_enhanced_test), y_test,digits=3))
y_pred = gnb.predict(df_enhanced_test)

              precision    recall  f1-score   support

           0      0.906     0.660     0.764     29036
           1      0.526     0.848     0.649     12965

    accuracy                          0.718     42001
   macro avg      0.716     0.754     0.706     42001
weighted avg      0.789     0.718     0.728     42001



### SVM

In [ ]:
# from sklearn.svm import SVC
# svc = SVC(C=1, kernel="rbf",gamma='auto')
# svc.fit(df_tfidf_train, y_train)
# print(classification_report(svc.predict(df_tfidf_test), y_test))

### Logestic Regression

In [11]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(df_enhanced_train, y_train)
print(classification_report(lr.predict(df_enhanced_test), y_test,digits=3))
y_pred = lr.predict(df_enhanced_test)

NameError: ignored

## RandomForest

In [38]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
clf = RandomForestClassifier(max_depth=7, random_state=0)
clf.fit(df_enhanced_train, y_train)
#df_tfidf_test
print(classification_report(clf.predict(df_enhanced_test), y_test,digits=3))
y_pred = clf.predict(df_enhanced_test)

              precision    recall  f1-score   support

           0      0.844     0.682     0.755     26125
           1      0.603     0.792     0.685     15876

    accuracy                          0.724     42001
   macro avg      0.723     0.737     0.720     42001
weighted avg      0.753     0.724     0.728     42001



In [ ]:
print(classification_report(clf.predict(df_tfidf_test), y_test))

              precision    recall  f1-score   support

           0       0.84      0.72      0.77     24643
           1       0.67      0.80      0.73     17358

    accuracy                           0.75     42001
   macro avg       0.75      0.76      0.75     42001
weighted avg       0.77      0.75      0.75     42001



## NN

In [41]:
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
NN = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(128, 2), random_state=1)
#LR = LogisticRegression(random_state=0).fit(X_train, Y_train)
NN.fit(df_enhanced_train, y_train)
#LR.fit(X_train, Y_train)
print(classification_report(NN.predict(df_enhanced_test), y_test))
y_pred = NN.predict(df_enhanced_test)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


              precision    recall  f1-score   support

           0       0.85      0.80      0.82     22521
           1       0.78      0.84      0.81     19480

    accuracy                           0.81     42001
   macro avg       0.81      0.82      0.81     42001
weighted avg       0.82      0.81      0.81     42001



In [ ]:
acc = 0
y_test=list(y_test)
for i in range(len(y_pred)):
  if y_pred[i]==y_test[i]:
    acc += 1
acc = acc/len(y_pred)
acc

0.8140282374229185

In [42]:
fpr, tpr, thresholds = metrics.roc_curve(list(y_test), list(y_pred), pos_label=1)
metrics.auc(fpr, tpr)

0.813823103012946

In [39]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

0.7233092236430292

In [15]:
from nltk.corpus import stopwords
list_dimensions = [1000,2000,3000,4000,5000,6000,7000]
for k in list_dimensions:
  en_stopwords = set(stopwords.words("english"))
  vector = TfidfVectorizer(max_features=k,stop_words=en_stopwords)
  train_tfidf = vector.fit_transform(X_train['review'].apply(lambda x: np.str_(x)))
  test_tfidf = vector.transform(X_test['review'].apply(lambda x: np.str_(x)))
  df_tfidf_train = pd.DataFrame(train_tfidf.todense())
  df_tfidf_test = pd.DataFrame(test_tfidf.todense())
  lr = LogisticRegression()
  lr.fit(df_tfidf_train, y_train)
  print(classification_report(lr.predict(df_tfidf_test), y_test,digits=3))

              precision    recall  f1-score   support

           0      0.866     0.814     0.839     22466
           1      0.800     0.855     0.827     19535

    accuracy                          0.833     42001
   macro avg      0.833     0.835     0.833     42001
weighted avg      0.835     0.833     0.833     42001



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


              precision    recall  f1-score   support

           0      0.875     0.832     0.853     22220
           1      0.821     0.867     0.844     19781

    accuracy                          0.849     42001
   macro avg      0.848     0.850     0.848     42001
weighted avg      0.850     0.849     0.849     42001



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


              precision    recall  f1-score   support

           0      0.879     0.839     0.858     22118
           1      0.830     0.871     0.850     19883

    accuracy                          0.854     42001
   macro avg      0.854     0.855     0.854     42001
weighted avg      0.855     0.854     0.854     42001



...

              precision    recall  f1-score   support

           0      0.881     0.845     0.863     22015
           1      0.837     0.874     0.855     19986

    accuracy                          0.859     42001
   macro avg      0.859     0.860     0.859     42001
weighted avg      0.860     0.859     0.859     42001



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


              precision    recall  f1-score   support

           0      0.881     0.847     0.864     21991
           1      0.839     0.875     0.856     20010

    accuracy                          0.860     42001
   macro avg      0.860     0.861     0.860     42001
weighted avg      0.861     0.860     0.860     42001



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


              precision    recall  f1-score   support

           0      0.882     0.847     0.864     21985
           1      0.839     0.875     0.857     20016

    accuracy                          0.861     42001
   macro avg      0.861     0.861     0.861     42001
weighted avg      0.862     0.861     0.861     42001



In [16]:
en_stopwords = set(stopwords.words("english"))
vector = TfidfVectorizer(max_features=2000,stop_words=en_stopwords)
train_tfidf = vector.fit_transform(X_train['review'].apply(lambda x: np.str_(x)))
test_tfidf = vector.transform(X_test['review'].apply(lambda x: np.str_(x)))
df_tfidf_train = pd.DataFrame(train_tfidf.todense())
df_tfidf_test = pd.DataFrame(test_tfidf.todense())
lr = LogisticRegression()
lr.fit(df_tfidf_train, y_train)
print(classification_report(lr.predict(df_tfidf_test), y_test,digits=3))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


              precision    recall  f1-score   support

           0      0.875     0.832     0.853     22220
           1      0.821     0.867     0.844     19781

    accuracy                          0.849     42001
   macro avg      0.848     0.850     0.848     42001
weighted avg      0.850     0.849     0.849     42001



In [18]:
y_pred = lr.predict(df_tfidf_test)
fpr, tpr, thresholds = metrics.roc_curve(list(y_test), list(y_pred), pos_label=1)
metrics.auc(fpr, tpr)

0.8483663425775543